In [2]:
! pip install mlflow transformers datasets "pydantic[dotenv]" av

In [2]:
%cd ..

/notebooks


In [4]:
!ls 

Makefile   gallery.md		      mlflow.db  overfit	 scripts
README.md  imagenet_classes.txt       mlruns	 poetry.lock	 sphere.png
data	   imagenet_map_classes.json  notebooks  pyproject.toml


In [5]:
import argparse
import logging
import os

import mlflow
from overfit.models.vit import ViT
from overfit.trainers.overfit import OverfitTrainer
from overfit.utils.misc import parse_video_path_params
from torchvision.io import read_video
from torchvision import transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torchvision.models import ResNet50_Weights, resnet50
from torchvision.models import ResNet34_Weights, resnet34
from torchvision.models import ResNet18_Weights, resnet18
from pathlib import Path
import torch
from tqdm.notebook import tqdm
from mlflow.client import MlflowClient
from overfit.env_settings import settings

In [6]:
DATASET = "4-50"
CONFIDENCE=0.1
WEIGHT_DECAY=0.2
MAX_LR=0.4
MOMENTUM=0.1
MODELS = ["resnet18", "resnet34", "resnet50", "vit"]
MODEL=MODELS[2]
EXPERIMENT_NAME = f"D{DATASET}M{MODEL}C{CONFIDENCE}WD{WEIGHT_DECAY}LR{MAX_LR}M{MOMENTUM}"

In [7]:
client = MlflowClient(tracking_uri=settings.MLFLOW_TRACKING_URI)
mlflow.set_tracking_uri(settings.MLFLOW_TRACKING_URI)
try:
  MLFLOW_EXPERIMENT_ID = client.create_experiment(EXPERIMENT_NAME)
except Exception as e:
  MLFLOW_EXPERIMENT_ID = client.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
print(MLFLOW_EXPERIMENT_ID, EXPERIMENT_NAME)

27 D4-50Mresnet50C0.1WD0.2LR0.4M0.1


In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [9]:
class ToFloat(object):
    def __init__(self):
      pass

    def __call__(self, tensor):
      return (tensor / 255.0).type(torch.float32)

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((224, 224)),
    ToFloat(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

In [10]:
!ls /datasets

imagenet1k-4-50  imagenet1k-5-50  imagenet1k-6-75


In [11]:
videos = [v for v in Path(f"/datasets/imagenet1k-{DATASET}").glob("*.mp4")]
videos[:2]

[PosixPath('/datasets/imagenet1k-4-50/0-tench-4-40.mp4'),
 PosixPath('/datasets/imagenet1k-4-50/1-goldfish-4-40.mp4')]

In [ ]:
with open("imagenet_classes.txt", "r") as f:
    categories = f.readlines()
    categories = [cat.rstrip("\n") for cat in categories]

logging.info("Creating trainer")
# srcnet = ViT(device=device).eval().to(device)
if MODEL == "vit":
    srcnet = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1).eval().to(device)
elif MODEL == "resnet34":
    srcnet = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1).eval().to(device)
elif MODEL == "resnet50":
    srcnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1).eval().to(device)
elif MODEL == "resnet8":
    srcnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).eval().to(device)
else:
    raise Exception("Unknown Source model")
for video_path in tqdm(videos):
  video_path = str(video_path)
  vid = read_video(video_path, output_format="TCHW")[0]
  vid = TRANSFORM_IMG(vid).to(device)
  y_ix, _, crop_fraction, n_frames = parse_video_path_params(video_path)
  logging.info(crop_fraction)
  logging.info(n_frames)

  tgtnet_trainer = OverfitTrainer(categories=categories)
  tgtnet_trainer.set(
      pretrained_classifier=srcnet,
      num_classes=1000,
      confidence=CONFIDENCE,
      weight_decay=WEIGHT_DECAY,
      max_lr=MAX_LR,
      momentum=MOMENTUM,
  )
  tgtnet_trainer.model = tgtnet_trainer.model.to(device)

  logging.info("Starting experiment")
  with mlflow.start_run(experiment_id=MLFLOW_EXPERIMENT_ID) as run:
      mlflow.log_param("Crop fraction", crop_fraction)
      mlflow.log_param("Frames", n_frames)
      mlflow.log_param("Filename", video_path)
      mlflow.log_param("Source Model", "Resnet50")
      tgtnet_trainer.test(vid, [y_ix] * n_frames, active_run=run, hf_format=False)